In [1]:
%%capture
!pip install transformers datasets
!pip install flax optaxjaxlib
!pip install numpy pandas
!pip install tqdm
!pip install scikit-learn
!pip install evaluate

In [2]:
pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html


In [3]:
import jax
print(jax.devices())

[CudaDevice(id=0)]


# Loading The Data


In [4]:
from datasets import Dataset
import pandas as pd
import os

def load_data_from_dir(dir_path, label):
    data = []
    for filename in sorted(os.listdir(dir_path)):
        file_path = os.path.join(dir_path, filename)

        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read().strip()
        except UnicodeDecodeError:
            with open(file_path, 'r', encoding='ISO-8859-9') as f:
                text = f.read().strip()

        data.append({'sentence': text, 'labels': label})
    return data

neg_dir   = "/content/sorted_news/neg"
pos_dir   = "/content/sorted_news/pos"
neg_data  = load_data_from_dir(neg_dir, 0)
pos_data  = load_data_from_dir(pos_dir, 1)
all_data  = neg_data + pos_data

seed = 35
full_dataset = Dataset.from_pandas(pd.DataFrame(all_data))
full_dataset = full_dataset.shuffle(seed = seed)

print(full_dataset[2:5])
# for i in full_dataset:
#    print(i)

train_test_split = full_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

print(train_dataset)
print(test_dataset)

{'sentence': ['Cuma günü tarihi zirvesini 4.552,44 seviyesine taşıyan BIST-100 Endeksi kapanışa doğru etkili olan kâr satışlarıyla gün içi kazançlarını geri verdi. Yaşanan güçlü yükselişlerin ardından son günlerde endekste ve ana hisselerde gözlenen yorulma emareleri ve teknik indikatörlerdeki negatif uyuşmazlıklar olası bir düzeltme ihtimalini artırıyor. Endekste kısa vadede 4.400 seviyesi kısa vadeli destek olarak izlenecek olup, bu seviye altında 4.333 \x96 4.297 \x96 4.234 - 4.150 ve 4.100 seviyeleri destek olarak takip edilebilir. Endekste kısa vadede 4.400 üzerinde kalıcılığın korunması yükselişlerin devamlılığı açısından önem taşımaktadır. Endekste 4.400 üzerindeki tutunmanın korunması ve 4.500 üzerinde kapanışların yaşanması durumunda ise 4.552 \x96 4.575 ve 4.600 seviyeleri direnç konumunda bulunmaktadır.', 'Haftaya global eğilime paralel sınırlı zayıf eğilimle başlangıç beklediğimiz  endekste ilk aşamada 3500-3490 destek, 3610-3630 direnç bölgelerinin son 6 işlem gününde  olu

# Tokenizing the Data

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('dbmdz/bert-base-turkish-cased')

def preprocess_function(examples):
    texts     = (examples['sentence'],)
    processed = tokenizer(*texts, padding="max_length", max_length=256, truncation=True)

    processed["labels"] = examples["labels"]
    return processed

train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset  = test_dataset.map(preprocess_function, batched=True)

train_dataset.set_format(type='np', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='np', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

print(train_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/1364 [00:00<?, ? examples/s]

Map:   0%|          | 0/342 [00:00<?, ? examples/s]

Dataset({
    features: ['sentence', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1364
})


# Loading the Model

In [6]:
from transformers import FlaxBertForSequenceClassification

id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

model = FlaxBertForSequenceClassification.from_pretrained(
    'dbmdz/bert-base-turkish-cased',
    num_labels = 2,
    id2label   = id2label,
    label2id   = label2id
)

Some weights of FlaxBertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: {('classifier', 'kernel'), ('classifier', 'bias')}
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Evaluating Before

In [7]:
import evaluate
from tqdm import tqdm
import numpy as np
from sklearn.metrics import confusion_matrix

def evaluate_model(model, eval_dataset):
    accuracy_metric  = evaluate.load('accuracy')
    precision_metric = evaluate.load('precision')
    recall_metric    = evaluate.load('recall')

    # for conf matrix
    all_predictions = []
    all_labels = []

    batch_size = 16

    num_batches  = len(eval_dataset) // batch_size
    progress_bar = tqdm(range(num_batches), desc="Evaluating")

    for i in progress_bar:
        batch = eval_dataset[i*batch_size:(i+1)*batch_size]

        input_ids      = jax.device_put(np.array(batch['input_ids']))
        attention_mask = jax.device_put(np.array(batch['attention_mask']))
        labels         = jax.device_put(np.array(batch['labels']))

        # Forward pass through the model
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits  = outputs.logits

        # Convert JAX predictions to numpy and get the predicted class
        predictions = np.argmax(logits, axis=-1)
        predictions = np.array(predictions)

        all_predictions.extend(predictions)
        all_labels.extend(labels)

        accuracy_metric.add_batch(predictions=predictions, references=labels)
        precision_metric.add_batch(predictions=predictions, references=labels)
        recall_metric.add_batch(predictions=predictions, references=labels)

    accuracy_result = accuracy_metric.compute()
    precision_result = precision_metric.compute()
    recall_result = recall_metric.compute()

    confusion_mat = confusion_matrix(all_labels, all_predictions)

    return {
        "accuracy": accuracy_result['accuracy'],
        "precision": precision_result['precision'],
        "recall": recall_result['recall'],
        "confusion_matrix": confusion_mat
    }

In [8]:
eval_results = evaluate_model(model, test_dataset)

print(f"Accuracy: {eval_results['accuracy']:.4f}")
print(f"Precision: {eval_results['precision']:.4f}")
print(f"Recall: {eval_results['recall']:.4f}")
print(f"Confusion Matrix:\n{eval_results['confusion_matrix']}")

Evaluating: 100%|██████████| 21/21 [00:19<00:00,  1.09it/s]

Accuracy: 0.4554
Precision: 0.7500
Recall: 0.0162
Confusion Matrix:
[[150   1]
 [182   3]]


In [9]:
def after_evaluate_model(model, params, eval_dataset):
    # Load evaluation metrics from the evaluate library
    accuracy_metric = evaluate.load('accuracy')
    precision_metric = evaluate.load('precision')
    recall_metric = evaluate.load('recall')

    # Lists to store all predictions and labels for confusion matrix calculation
    all_predictions = []
    all_labels = []

    batch_size = 16  # Define your batch size

    num_batches = len(eval_dataset) // batch_size
    progress_bar = tqdm(range(num_batches), desc="Evaluating")

    for i in progress_bar:
        # Slice the dataset into batches
        batch = eval_dataset[i * batch_size:(i + 1) * batch_size]

        # Move data to JAX devices
        input_ids = jax.device_put(np.array(batch['input_ids']))
        attention_mask = jax.device_put(np.array(batch['attention_mask']))
        labels = jax.device_put(np.array(batch['labels']))

        # Forward pass through the model using the trained parameters
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, params=params, train=False)
        logits = outputs.logits

        # Convert logits to class predictions (0 or 1)
        predictions = np.argmax(logits, axis=-1)
        predictions = np.array(predictions)

        # Append predictions and labels for confusion matrix calculation
        all_predictions.extend(predictions)
        all_labels.extend(labels)

        # Add batches to the evaluation metrics
        accuracy_metric.add_batch(predictions=predictions, references=labels)
        precision_metric.add_batch(predictions=predictions, references=labels)
        recall_metric.add_batch(predictions=predictions, references=labels)

    # Compute evaluation results
    accuracy_result = accuracy_metric.compute()
    precision_result = precision_metric.compute()
    recall_result = recall_metric.compute()

    # Compute confusion matrix
    confusion_mat = confusion_matrix(all_labels, all_predictions)

    # Return the evaluation results
    return {
        "accuracy": accuracy_result['accuracy'],
        "precision": precision_result['precision'],
        "recall": recall_result['recall'],
        "confusion_matrix": confusion_mat
    }

# Training

In [10]:
import optax  # Optimizer library for JAX
import jax
from flax.training import train_state

learning_rate = 2e-5
tx = optax.adamw(learning_rate=learning_rate)

state = train_state.TrainState.create(
    apply_fn=model.__call__,
    params=jax.device_put(model.params),
    tx=tx)


In [11]:
def train_step(state, batch, rng):
    """Performs a training step."""

    def loss_fn(params):
        logits = model(input_ids=batch['input_ids'],
                       attention_mask=batch['attention_mask'],
                       params=params,
                       dropout_rng=rng,
                       train=True).logits
        loss = optax.softmax_cross_entropy_with_integer_labels(logits, batch['labels'])
        print(loss.mean)
        return loss.mean()

    # Compute gradients
    grad_fn = jax.value_and_grad(loss_fn)
    loss, grads = grad_fn(state.params)

    state = state.apply_gradients(grads=grads)

    return state, loss


In [12]:
from tqdm import tqdm

def train_model(state, train_dataset, num_epochs=3, batch_size=16):
    num_batches = len(train_dataset) // batch_size
    rng = jax.random.PRNGKey(0)

    rng, new_rng = jax.random.split(rng)
    rng = jax.device_put(rng)

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        progress_bar = tqdm(range(num_batches), desc="Training")

        for i in progress_bar:
            batch = train_dataset[i*batch_size:(i+1)*batch_size]
            batch = {k: jax.device_put(v) for k, v in batch.items()}

            rng, new_rng = jax.random.split(rng)
            state, loss = train_step(state, batch, new_rng)

            progress_bar.set_postfix({"loss": loss.item()})

        print(f"Evaluating after epoch {epoch + 1}...")
        eval_results = after_evaluate_model(model, state.params, test_dataset)

        # Print evaluation results
        print(f"Accuracy: {eval_results['accuracy']:.4f}")
        print(f"Precision: {eval_results['precision']:.4f}")
        print(f"Recall: {eval_results['recall']:.4f}")
        print(f"Confusion Matrix:\n{eval_results['confusion_matrix']}")

    return state

In [13]:
state = train_model(state, train_dataset)

Epoch 1/3


Training:   0%|          | 0/85 [00:00<?, ?it/s]

<bound method _forward_method_to_aval.<locals>.meth of Traced<ConcreteArray([0.47242653 1.1312478  0.39314756 0.7948002  0.63561356 0.9901922
 0.34019652 1.3354644  1.550159   0.932637   1.4192448  1.0556989
 0.85567945 0.8104117  1.2469792  1.299995  ], dtype=float32)>with<JVPTrace(level=2/0)> with
  primal = Array([0.47242653, 1.1312478 , 0.39314756, 0.7948002 , 0.63561356,
       0.9901922 , 0.34019652, 1.3354644 , 1.550159  , 0.932637  ,
       1.4192448 , 1.0556989 , 0.85567945, 0.8104117 , 1.2469792 ,
       1.299995  ], dtype=float32)
  tangent = Traced<ShapedArray(float32[16])>with<JaxprTrace(level=1/0)> with
    pval = (ShapedArray(float32[16]), None)
    recipe = JaxprEqnRecipe(eqn_id=<object object at 0x7c31a196d250>, in_tracers=(Traced<ShapedArray(float32[16]):JaxprTrace(level=1/0)>, Traced<ShapedArray(float32[16]):JaxprTrace(level=1/0)>), out_tracer_refs=[<weakref at 0x7c31a1a7f6f0; to 'JaxprTracer' at 0x7c31a1a7fba0>], out_avals=[ShapedArray(float32[16])], primitive=pjit,

Training:   1%|          | 1/85 [00:12<16:52, 12.06s/it, loss=0.954]

<bound method _forward_method_to_aval.<locals>.meth of Traced<ConcreteArray([0.55466515 0.4072841  0.6829692  0.5432238  1.0124465  0.2603053
 0.89486057 1.0057849  1.0634753  0.7047622  0.7201892  0.73057795
 0.54509234 1.0175405  1.419709   0.87405515], dtype=float32)>with<JVPTrace(level=2/0)> with
  primal = Array([0.55466515, 0.4072841 , 0.6829692 , 0.5432238 , 1.0124465 ,
       0.2603053 , 0.89486057, 1.0057849 , 1.0634753 , 0.7047622 ,
       0.7201892 , 0.73057795, 0.54509234, 1.0175405 , 1.419709  ,
       0.87405515], dtype=float32)
  tangent = Traced<ShapedArray(float32[16])>with<JaxprTrace(level=1/0)> with
    pval = (ShapedArray(float32[16]), None)
    recipe = JaxprEqnRecipe(eqn_id=<object object at 0x7c31a15196e0>, in_tracers=(Traced<ShapedArray(float32[16]):JaxprTrace(level=1/0)>, Traced<ShapedArray(float32[16]):JaxprTrace(level=1/0)>), out_tracer_refs=[<weakref at 0x7c31a1a2f2e0; to 'JaxprTracer' at 0x7c31a1a2cdb0>], out_avals=[ShapedArray(float32[16])], primitive=pjit

...

<bound method _forward_method_to_aval.<locals>.meth of Traced<ConcreteArray([0.11868417 0.00444175 0.01744053 0.04708511 0.06250921 0.01929958
 0.8214077  0.0372411  0.10214975 0.02849609 0.01699023 0.09417078
 1.2994682  0.01295389 0.00976028 0.01452715], dtype=float32)>with<JVPTrace(level=2/0)> with
  primal = Array([0.11868417, 0.00444175, 0.01744053, 0.04708511, 0.06250921,
       0.01929958, 0.8214077 , 0.0372411 , 0.10214975, 0.02849609,
       0.01699023, 0.09417078, 1.2994682 , 0.01295389, 0.00976028,
       0.01452715], dtype=float32)
  tangent = Traced<ShapedArray(float32[16])>with<JaxprTrace(level=1/0)> with
    pval = (ShapedArray(float32[16]), None)
    recipe = JaxprEqnRecipe(eqn_id=<object object at 0x7c31a1b86f20>, in_tracers=(Traced<ShapedArray(float32[16]):JaxprTrace(level=1/0)>, Traced<ShapedArray(float32[16]):JaxprTrace(level=1/0)>), out_tracer_refs=[<weakref at 0x7c31a1bd8310; to 'JaxprTracer' at 0x7c31a1bdbe20>], out_avals=[ShapedArray(float32[16])], primitive=pji

Training: 100%|██████████| 85/85 [03:58<00:00,  2.80s/it, loss=0.169]


Evaluating after epoch 3...


Evaluating: 100%|██████████| 21/21 [00:12<00:00,  1.73it/s]

Accuracy: 0.8631
Precision: 0.8840
Recall: 0.8649
Confusion Matrix:
[[130  21]
 [ 25 160]]


# Final Evaluation

In [14]:
eval_results = after_evaluate_model(model, state.params, test_dataset)

# Print the evaluation results
print(f"Accuracy: {eval_results['accuracy']:.4f}")
print(f"Precision: {eval_results['precision']:.4f}")
print(f"Recall: {eval_results['recall']:.4f}")
print(f"Confusion Matrix:\n{eval_results['confusion_matrix']}")

Evaluating: 100%|██████████| 21/21 [00:12<00:00,  1.72it/s]

Accuracy: 0.8631
Precision: 0.8840
Recall: 0.8649
Confusion Matrix:
[[130  21]
 [ 25 160]]
